### Estadisticas
#### 1 sample 1 epoch gpu 1
tarda de media 30 segundos.
#### 2 sample 1 epoch gpu 1
tarda de media 58 segundos.
#### 2 sample 1 epoch gpu 0.5
tarda de media 30 segundos. (y la gpu esta al 44% y le sobra memoria.)
#### 5 sample 1 epoch gpu 0.2
ocurren errores por memoria insuficiente en la ram (15.4 disponible para uso, del programa, y no alcanza.)
#### 4 sample 1 epoch gpu 0.25
tarda de media 1 minuto con 13 segundos, utiliza 11 gb de memoria ram, la gpu esta a mitad de capacidad de computo.
la cantidad de memoria ram es un cuello de botella para utilizar todo el poder de la GPU.
#### 4 sample 10 epoch gpu 0.25
tarda de media 2 minutos con 11 segundos, pero hay que tener cuidado ya que mientras entrena consume poca memoria, pero al momento de validar su loss y acc incrementa al nivel de warning con la RAM.
#### 3 sample 10 epoch gpu 0.33333
tarda 1 minuto y 40 segundos, pero con redes grandes tambien llega al warning de RAM.
#### 2 sample 10 epoch gpu 0.5
58 segundos, no se acerca a warning de RAM.
#### 10 sample 10 epoch gpu 0.5
4 minutos 25 segundos, corriendo de a dos modelos a la vez, maxima cantidad de memoria RAM usada 12.5 GB

2 configuraciones llegaron hasta las 10 epoch, 1 hasta la octava, el resto tuvo un desempeño pobre  desde el inicio y ray tune (programador ASHA) los termino en la primera epoch (se puede configurar la cantidad minima de epoch de ejecucion antes de terminarlas).

### Probar buscar hiperparametros de la red.
se deja estaticos el learning rate y el batch size. a 1-e4 y 16, respectivamente.

El resto se prueba buscar, para ver el tiempo y rendimiento se prueba 10 samples 50 epoch gpu 0.5

el resultado es una media de 14 minutos.


In [1]:
# python
import os
from functools import partial
# pytorch
import torch
from torch import nn
# pytorch utilidades
from torch.utils.data import random_split
# ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
# yo
from cargar_datos import cargar_datasets, cargar_dataloaders
from modelo_convolucional import instanciarRed


def main(num_samples=10, max_num_epochs=10, gpus_per_trial=0):
    data_dir = os.path.abspath("./data")
    checkpoint_dir = os.path.abspath("./checkpoint_dir")

    # se carga una vez, para que si no esta descargado, se descarge y se valide, el resto de veces en las pruebas, se saltara esta parte.
    cargar_datasets(data_dir)
    """
    variable de configuracion, Ray tune modificara sus valores en el proceso de ajuste de hiperparametros.
    """
    config = {
        #"batch_size": tune.choice([8, 16, 32]),
        #"learning_rate": tune.loguniform(1e-4, 1e-1),
        "batch_size": 16,
        "learning_rate": 1e-4,
        # red convolucional
        "cant_filtros_conv1": tune.choice([6, 12, 18]),
        "kernel_size_maxpool1": tune.choice([2, 3]),
        "cant_filtros_conv2": tune.choice([16, 22, 28]),
        "kernel_size_maxpool2": tune.choice([2, 3]),
        "full_l1": tune.choice([120, 140, 160]),
        "full_l2": tune.choice([84, 104, 124])
    }
    """
    configurar Ray y correr las pruebas en busqueda de hiperparametros
    """
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        parameter_columns=list(config.keys()),
        metric_columns=["loss", "accuracy", "training_iteration"])
    result = tune.run(
        # partial es una funcion de orden superior de tipo curry
        partial(train_cifar, data_dir=data_dir, checkpoint_dir=checkpoint_dir, max_num_epochs=max_num_epochs),
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

    """
    Validar el desempeño de la red con mejor puntuacion en la busqueda de hiperparametros contra el conjunto de datos de validacion.
    """
    best_trained_model = instanciarRed(best_trial.config)
    device = getDevice()
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.dir_or_data
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_acc = test_accuracy(best_trained_model, device, data_dir)
    print("Best trial test set accuracy: {}".format(test_acc))


def test_accuracy(model, device="cpu", data_dir="./data"):
    _, testset = cargar_datasets(data_dir)
    testloader = cargar_dataloaders(testset, 4)

    correct = 0
    total = 0
    # pone a la red en moodo evaluacion, desactiva capas dropout.
    model.eval()
    # desactiva el proceso de calculo y guardado de valores intermedios
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total


def getDevice():
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            raise Exception(
                "la pc cuenta con multiples gpus, deberia utilizar DataParallel")
            #model = nn.DataParallel(model)
    return device


def train_cifar(config, checkpoint_dir=None, data_dir=None,max_num_epochs=10) :
    model = instanciarRed(config)

    device = getDevice()
    # le dice a Pytorch donde debe ejecutar el modelo.
    model.to(device)
    # CrossEntropyLoss requiere el output "logits", no es necesario pasarlo por el softmax, ya que lo calcula dentro. https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
    criterion = nn.CrossEntropyLoss()

    # se necesita enviar los parametros del modelo al optimizador para que los pueda actualizar.
    optimizer = torch.optim.Adam(
        model.parameters(), lr=config["learning_rate"])

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    trainset, _ = cargar_datasets(data_dir)
    # reservar una parte de los datos de entrenamiento para validacion.
    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])

    trainloader = cargar_dataloaders(train_subset, config["batch_size"])
    valloader = cargar_dataloaders(val_subset, config["batch_size"])

    for epoch in range(max_num_epochs):  # loop over the dataset multiple times

        train_loop(model, device, trainloader, criterion, optimizer, epoch)
        loss, accuracy = validation_loss(model, device, valloader, criterion)

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=loss, accuracy=accuracy)
    print("Finished Training")


def train_loop(model, device, trainloader, criterion, optimizer, epoch):
    model.train()
    running_loss = 0.0
    epoch_steps = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        epoch_steps += 1
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                            running_loss / epoch_steps))
            running_loss = 0.0


def validation_loss(net, device, valloader, criterion):
    val_loss = 0.0
    val_steps = 0
    total = 0
    correct = 0
    net.eval()
    for i, data in enumerate(valloader, 0):

        with torch.no_grad():
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            loss = criterion(outputs, labels)
            val_loss += loss.cpu().numpy()
            val_steps += 1
    loss = val_loss / val_steps
    accuracy = correct / total
    return loss, accuracy


if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    main(num_samples=10, max_num_epochs=50, gpus_per_trial=0.5)


Files already downloaded and verified
Files already downloaded and verified


2022-11-16 23:31:27,286	INFO worker.py:1519 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
/home/braian/miniconda3/envs/tesis/lib/python3.9/site-packages/ray/tune/trainable/function_trainable.py:609: DeprecationWarning: `checkpoint_dir` in `func(config, checkpoint_dir)` is being deprecated. To save and load checkpoint in trainable functions, please use the `ray.air.session` API:

from ray.air import session

def train(config):
    # ...
    session.report({"metric": metric}, checkpoint=checkpoint)

For more information please see https://docs.ray.io/en/master/tune/api_docs/trainable.html

  warnings.warn(


== Status ==
Current time: 2022-11-16 23:31:28 (running for 00:00:00.17)
Memory usage on this node: 4.0/15.4 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/20 CPUs, 0.5/1 GPUs, 0.0/7.87 GiB heap, 0.0/3.94 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /home/braian/ray_results/train_cifar_2022-11-16_23-31-27
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+---------------------+--------------+-----------------+----------------------+------------------------+----------------------+------------------------+-----------+-----------+
| Trial name              | status   | loc                 |   batch_size |   learning_rate |   cant_filtros_conv1 |   kernel_size_maxpool1 |   cant_filtros_conv2 |   kernel_size_maxpool2 |   full_l1 |   full_l2 |
|-------------------------+----------+---------------------+-----

0.0%(func pid=25614) 
0.0%(func pid=25588) 
0.0%(func pid=25614) 
0.0%(func pid=25588) 
0.1%(func pid=25614) 
0.1%(func pid=25588) 
0.2%(func pid=25614) 
0.2%(func pid=25588) 
0.5%(func pid=25614) 
0.5%(func pid=25588) 
1.0%(func pid=25614) 
1.1%(func pid=25588) 
2.0%(func pid=25614) 
2.1%(func pid=25614) 
2.2%(func pid=25614) 
2.2%(func pid=25614) 
1.7%(func pid=25588) 
3.4%(func pid=25614) 
1.7%(func pid=25588) 
1.7%(func pid=25588) 
3.5%(func pid=25614) 
2.2%(func pid=25588) 
4.0%(func pid=25614) 
2.3%(func pid=25588) 
4.5%(func pid=25614) 
2.3%(func pid=25588) 
4.6%(func pid=25614) 
3.9%(func pid=25588) 
5.8%(func pid=25614) 
5.8%(func pid=25614) 
5.9%(func pid=25614) 
4.0%(func pid=25588) 
6.4%(func pid=25614) 
4.1%(func pid=25588) 
4.2%(func pid=25588) 
5.5%(func pid=25588) 
7.5%(func pid=25614) 
5.6%(func pid=25588) 
5.7%(func pid=25588) 
7.5%(func pid=25614) 
5.7%(func pid=25588) 
7.6%(func pid=25614) 
7.6%(func pid=25614) 
7.6%(func pid=25614) 
7.7%(func pid=25614) 
7.7%(func 

== Status ==
Current time: 2022-11-16 23:31:44 (running for 00:00:16.22)
Memory usage on this node: 6.0/15.4 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 4.0/20 CPUs, 1.0/1 GPUs, 0.0/7.87 GiB heap, 0.0/3.94 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /home/braian/ray_results/train_cifar_2022-11-16_23-31-27
Number of trials: 10/10 (8 PENDING, 2 RUNNING)
+-------------------------+----------+---------------------+--------------+-----------------+----------------------+------------------------+----------------------+------------------------+-----------+-----------+
| Trial name              | status   | loc                 |   batch_size |   learning_rate |   cant_filtros_conv1 |   kernel_size_maxpool1 |   cant_filtros_conv2 |   kernel_size_maxpool2 |   full_l1 |   full_l2 |
|-------------------------+----------+---------------------+-----

...

== Status ==
Current time: 2022-11-16 23:44:50 (running for 00:13:22.37)
Memory usage on this node: 8.2/15.4 GiB 
Using AsyncHyperBand: num_stopped=8
Bracket: Iter 32.000: -1.0786221107959748 | Iter 16.000: -1.210127835559845 | Iter 8.000: -1.3280340267658235 | Iter 4.000: -1.4736945724964143 | Iter 2.000: -1.5981393854141235 | Iter 1.000: -1.7369934560775757
Resources requested: 4.0/20 CPUs, 1.0/1 GPUs, 0.0/7.87 GiB heap, 0.0/3.94 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /home/braian/ray_results/train_cifar_2022-11-16_23-31-27
Number of trials: 10/10 (2 RUNNING, 8 TERMINATED)
+-------------------------+------------+----------------------+--------------+-----------------+----------------------+------------------------+----------------------+------------------------+-----------+-----------+----------+------------+----------------------+
| Trial name              | status     | loc                  |   batch_size |   learning_rate |   cant_filtros_conv1 |   kernel_size_max

90.5%(func pid=113808) 
90.6%(func pid=113808) 
90.6%(func pid=113808) 
91.7%(func pid=113808) 
92.6%(func pid=113808) 
93.4%(func pid=113808) 
94.4%(func pid=113808) 
95.4%(func pid=113808) 
96.2%(func pid=113808) 
97.0%(func pid=113808) 
97.9%(func pid=113808) 
98.7%(func pid=113808) 
99.6%(func pid=113808) 
100.0%36m(func pid=113808) 


(func pid=113808) Extracting ./data/cifar-10-python.tar.gz to ./data
(func pid=113808) Files already downloaded and verified
(func pid=90312) [39,  2000] loss: 0.805
== Status ==
Current time: 2022-11-16 23:44:55 (running for 00:13:27.37)
Memory usage on this node: 10.8/15.4 GiB 
Using AsyncHyperBand: num_stopped=8
Bracket: Iter 32.000: -1.0786221107959748 | Iter 16.000: -1.210127835559845 | Iter 8.000: -1.3280340267658235 | Iter 4.000: -1.4736945724964143 | Iter 2.000: -1.5981393854141235 | Iter 1.000: -1.7369934560775757
Resources requested: 4.0/20 CPUs, 1.0/1 GPUs, 0.0/7.87 GiB heap, 0.0/3.94 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /home/braian/ray_results/train_cifar_2022-11-16_23-31-27
Number of trials: 10/10 (2 RUNNING, 8 TERMINATED)
+-------------------------+------------+----------------------+--------------+-----------------+----------------------+------------------------+----------------------+------------------------+-----------+-----------+----------+-------

2022-11-16 23:45:49,097	INFO tune.py:777 -- Total run time: 861.15 seconds (861.03 seconds for the tuning loop).


== Status ==
Current time: 2022-11-16 23:45:48 (running for 00:14:21.04)
Memory usage on this node: 6.9/15.4 GiB 
Using AsyncHyperBand: num_stopped=10
Bracket: Iter 32.000: -1.0786221107959748 | Iter 16.000: -1.210127835559845 | Iter 8.000: -1.3280340267658235 | Iter 4.000: -1.4736945724964143 | Iter 2.000: -1.5981393854141235 | Iter 1.000: -1.751924994659424
Resources requested: 0/20 CPUs, 0/1 GPUs, 0.0/7.87 GiB heap, 0.0/3.94 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /home/braian/ray_results/train_cifar_2022-11-16_23-31-27
Number of trials: 10/10 (10 TERMINATED)
+-------------------------+------------+----------------------+--------------+-----------------+----------------------+------------------------+----------------------+------------------------+-----------+-----------+----------+------------+----------------------+
| Trial name              | status     | loc                  |   batch_size |   learning_rate |   cant_filtros_conv1 |   kernel_size_maxpool1 |   cant